# Data Cleaning and Preparation

In [1]:
import numpy as np
import pandas as pd
import time
import datetime

import geopandas as gpd

import plotly.express as px
import plotly.graph_objects as go

In [2]:
for i in os.listdir('../datasets/raw'):
    print(i)

databoks-aug-2022.xlsx
kemenaker-2018-2022.xlsx
upah-minimum-2023.xlsx
upah-minimum-provinsi-raw-bps.csv


We have multiple datasets with multiple formats. Let's dig each one of them.

In [8]:
# load dataframes
df1 = pd.read_excel("../datasets/raw/databoks-aug-2022.xlsx")
df2 = pd.read_excel("../datasets/raw/kemenaker-2018-2022.xlsx")
df3 = pd.read_excel("../datasets/raw/upah-minimum-2023.xlsx")
df4 = pd.read_csv("../datasets/raw/upah-minimum-provinsi-raw-bps.csv", delimiter=";")

# display shape
print("df1's shape: ", df1.shape)
print("df2's shape: ", df2.shape)
print("df3's shape: ", df3.shape)
print("df4's shape: ", df4.shape)

df1's shape:  (35, 2)
df2's shape:  (34, 6)
df3's shape:  (34, 2)
df4's shape:  (34, 21)


display first five rows for every dataframe

In [4]:
df1.head()

,nama_data,2022
0,DKI Jakarta,4641854
1,Papua,3561932
2,Sulawesi Utara,3310723
3,Kep. Bangka Belitung,3264884
4,Papua Barat,3200000


In [5]:
df2.head()

,Provinsi,2018,2019,2020,2021,2022*
0,Aceh,2700000.00,2916810.00,3165031.00,3165031.00,3166460.00
1,Sumatera Utara,2132188.68,2303403.43,2499423.06,2499423.06,2522609.94
2,Sumatera Barat,2119067.00,2289220.00,2484041.00,2484041.00,2512539.00
3,Riau,2464154.06,2662025.63,2888584.01,2888564.01,2938564.01
4,Jambi,2243718.56,2423889.16,2630162.13,2630162.13,2698940.87


In [6]:
df3.head()

,nama_data,2023
0,DKI Jakarta,4901798
1,Papua,3864696
2,Kep. Bangka Belitung,3498479
3,Sulawesi Utara,3485000
4,Aceh,3413666


In [10]:
df4.head()

,provinsi,2020,2019,2018,2016,2015,2014,2013,2012,2011,...,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
0,ACEH,3165031,2916810,2700000,2118500,1900000,1750000,1550000,1400000,1350000,...,1200000,1000000,850000,820000,620000,550000,425000,330000,300000,265000
1,SUMATERA UTARA,2499423,2303403,2132189,1811875,1625000,1505850,1375000,1200000,1035500,...,905000,822205,761000,737794,600000,537000,505000,464000,340500,254000
2,SUMATERA BARAT,2484041,2289220,2119067,1800725,1615000,1490000,1350000,1150000,1055000,...,880000,800000,725000,650000,540000,480000,435000,385000,250000,200000
3,RIAU,2888564,2662026,2464154,2095000,1878000,1700000,1400000,1238000,1120000,...,901600,800000,710000,637000,551500,476900,437500,394000,329000,250700
4,JAMBI,2630162,2423889,2243719,1906650,1710000,1502300,1300000,1142500,1028000,...,800000,724000,658000,563000,485000,425000,390000,304000,245000,173000


Dataframes description:
- The 2022 minimum wage in `df2` its only temporary (Feb 2022) and it needs to be replaced by the minimum wage in `df1` since its the latest update (Aug 2022).
- `df3` contains next year mininum wage (2023).
- `df4` contains historical minimum wage for every provinces from 2000 to 2020. 
- Needs to combined 2021 and 2022 minimum wage using the merged dataframes between `df1` and `df2`, and then add minimum wage 2023 using the data from `df3`.